In [1]:
# Package Imports
import pandas as pd

In [2]:
# Read in book data
colnames = ['book_id','title','avg_rating','description']
users = pd.read_csv('books.csv', names=colnames)
# Isolate book_id
users = users[['book_id']]
# Make empty users set
users = users.transpose()
users = users.rename(columns=users.iloc[0]).drop(users.index[0])
users.insert(0,'user_id','')
users

,user_id,12182387,20135365,21401181,10099492,22642971,31556136,8030991,12182388,9684424,...,18068382,22521638,13536362,9568989,16373114,18221503,8987191,1885730,23636536,17805813


In [3]:
# Read in interactions data
colnames = ['user_id','book_id','is_read','rating']
interactions = pd.read_csv('interactions.csv', names=colnames)
interactions = interactions[interactions['is_read']==True]
interactions

,user_id,book_id,is_read,rating
3,8842281e1d1347389f2ab93d60773d4d,8684868,True,3
4,8842281e1d1347389f2ab93d60773d4d,8423493,True,2
5,8842281e1d1347389f2ab93d60773d4d,87976,True,5
6,8842281e1d1347389f2ab93d60773d4d,18116,True,5
7,8842281e1d1347389f2ab93d60773d4d,2767052,True,5
...,...,...,...,...
34919249,b0883ebf8e31731f1c5d91e678c26666,19057,True,5
34919250,b0883ebf8e31731f1c5d91e678c26666,13335037,True,4
34919251,b0883ebf8e31731f1c5d91e678c26666,1295102,True,4
34919252,b0883ebf8e31731f1c5d91e678c26666,11870085,True,5


In [4]:
# Read in review data
colnames = ['user_id','book_id','review_text','rating']
reviews = pd.read_csv('reviews.csv', names=colnames)
reviews

,user_id,book_id,review_text,rating
0,8842281e1d1347389f2ab93d60773d4d,2767052,I cracked and finally picked this up. Very enj...,5
1,7504b2aee1ecb5b2872d3da381c6c91e,23302416,I read this book because my fifth grade son wa...,5
2,f8a89075dc6de14857561522e729f82c,18053080,Though the book started out slow and only star...,4
3,f8a89075dc6de14857561522e729f82c,17383543,"*Update - 10/27/13* - After some sleep, I thin...",2
4,f8a89075dc6de14857561522e729f82c,16651458,"This is a moving, heartbreaking, view into a l...",5
...,...,...,...,...
2389895,a13e16a58266ffa4a0851c417c4cda8f,6137154,A really amazing book! It goes great with Grac...,5
2389896,fc5aea6994fc754daefa27cbea10ac79,10193062,This book is better then the first. It has it'...,3
2389897,fc5aea6994fc754daefa27cbea10ac79,6186357,I think this may have come from having expecta...,2
2389898,fc5aea6994fc754daefa27cbea10ac79,14740456,I generally liked the book. It had some parts ...,4


In [ ]:
# THIS TAKES TOO LONG TO RUN

# Group data by user_id
grouped = interactions.groupby('user_id')

# Add row for each user id
for name, group in grouped:
    users.loc[len(users)] = 0
    users.iloc[len(users)-1,0] = name
    for book, rate in zip(group.book_id, group.rating):
        users.loc[len(users)-1,book] = rate
users

In [ ]:
# Export to CSV
users.to_csv('users.csv', index=False)